In [1]:
using JuMP
using Ipopt

In [2]:
# scales: tons, years, MXNia, hours, trips
tmax = 27 # model run, years
n1 = -22.239 # ML, slope
n2 = 49.811 # ML, intersect
l1 = -0.0028 # q, slope
l2 = 0.1667 # q, intersect
a1 = 1/3.4E7 # proportion of migrating squid, where 3.4E7 max(e^(tau-b1))
K = 1208770 # carrying capacity in t
f = 40 # l of fuel per trip
B_h = 7.203 # hours per fisher
B_f = 2 # fisher per panga
h1 = 2E-10 # scale E
h2 = 0.6596 # scale E


### Variables ##################################################################
q = Array{Float64}(tmax); # catchability squid population
ML = Array{Float64}(tmax); # mantle length
y_S = Array{Float64}(tmax); # distance of squid migration from initial fishing grounds
R_tt = Array{Float64}(tmax); # trader cooperation
S = Array{Float64}(tmax); # size of the squid population
c_t = Array{Float64}(tmax); # cost of transport
Escal = Array{Float64}(tmax); # scale effort
E = Array{Float64}(tmax); # fishing effort
p_escal = Array{Float64}(tmax); # export price
p_min = Array{Float64}(tmax); # minimum wage
R = Array{Float64}(tmax); # revenue of fishers

### Initial values #############################################################
q[1] = 0.01 # squid catchability
y_S[1] = 0.5 # proportion of migrated squid
R_tt[1] = 0.5 # trader cooperation
S[1] = 1208770 # size of the squid population
c_t[1] = 5492603.58*1.4 #m *g # fleet cost of transport
E[1] = 1. # fishing effort
R[1] = 120877. *15438. -(c_t[1] + E[1]) #C1*p_f-

1.858409479988e9

In [3]:
using DataFrames
import XLSX
df1 = dropmissing(DataFrame(XLSX.readtable("./DATA/R3_data.xlsx", "Sheet1")...))
Missings.replace
# load columns
y = df1[:year] #
pe = df1[:pe_MXNiat] #
pf = df1[:pf_MXNiat] #
ct = df1[:C_t] #
ssh = df1[:essh_avg] #
ml = df1[:ML] #
ys = df1[:y_S] #

### New max time
tmax = length(y)

13

In [4]:
model = JuMP.Model(solver = IpoptSolver(print_level=3, max_iter=99900,print_frequency_iter=50,sb="yes"));

In [5]:
@variable(model, 38.750 <= b1 <= 42.1, start = 41.750) # isotherm depth
@variable(model, -6.9 <= b2 <= -3.987, start = -5.696) # isotherm depth
@variable(model, 11.478 <= b3 <= 16.4, start = 16.397) # isotherm depth
@variable(model, 0.01 <= beta <= 0.1, start = 0.0736) # slope of demand-price function
@variable(model, 1000.0 <= c_p <= 2148, start = 1776.25) # cost of processing, MXNia/t
@variable(model, 0.0 <= g <= 2.9, start = 1.4) # population growth rate
@variable(model, 20000.0 <= gamma <= 51000, start = 49200) # maximum demand, t
@variable(model, 2368793.0 <= m <= 8450159, start = 5492603.58) # cost per unit of transport all boats, MXN/trip
@variable(model, 11956952.0 <= w_m <= 28108539, start = 13355164) # min wage per hour all fleet

@variable(model, tau[t=1:tmax]) # temperature
@variable(model, p_e[t=1:tmax]) # export price
@variable(model, p_f[t=1:tmax]) # price for fishers
@variable(model, C[t=1:tmax]) # squid catch TODO: constraint

@variable(model, data[t=1:tmax])
@variable(model, match)

@constraint(model, [t=1:tmax], tau[t] == b1+b2*cos.(t)+b3*sin.(t));
@NLconstraint(model, [t=1:tmax], p_e[t] == gamma*(C[t])^(-beta));
@constraint(model, [t=1:tmax], p_f[t] == p_e[t] - c_p);
@NLconstraint(model, match == sum(abs(p_f[t] - data[t]) for t in 1:tmax));

# Initial conditions
@constraint(model, tau[1] == 42.);
@constraint(model, p_f[1] == 15438.); # max p_f datamares
@constraint(model, p_e[1] == 164706.); # max p_e comtrade
@constraint(model, C[1] == 120877.); # squid catch
@constraint(model, data[t=1:tmax] .== pf[t])

@NLobjective(model, Min, match);

In [6]:
solve(model)

:Error

Total number of variables............................:       75
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        9
                     variables with only upper bounds:        0
Total number of equality constraints.................:       57
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0


Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 1
Number of equality constraint evaluations            = 1
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations           